목적 : listed data Modeling from standard scaled 
===================

import modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

read csv files

In [3]:
train = pd.read_csv('../datasets/Train_Test/listed_resampled_train_data.csv')
test = pd.read_csv('../datasets/Train_Test/listed_test_data.csv')

In [4]:
# train_life_cycle = train[['도입기', '성장기', '성숙기', '쇠퇴기']]
# test_life_cycle = test[['도입기', '성장기', '성숙기', '쇠퇴기']]

selected features

In [5]:
x_train = train[['CASH FLOW 대 부채비율', '당좌비율', '순운전자본비율', '자기자본구성비율','경영자본순이익률',
                 '총자본영업이익률', '매출액영업이익률', '금융비용부담률', '이윤분배율', '유형자산회전율',
                 '상장년수', 'PCR', '쭈피처', '도입기', '성장기', '성숙기', '쇠퇴기']]

x_test = test[['CASH FLOW 대 부채비율', '당좌비율', '순운전자본비율', '자기자본구성비율','경영자본순이익률',
                 '총자본영업이익률', '매출액영업이익률', '금융비용부담률', '이윤분배율', '유형자산회전율',
                 '상장년수', 'PCR', '쭈피처', '도입기', '성장기', '성숙기', '쇠퇴기']]

y_train = train['부실판단']
y_test = test['부실판단']

# describe

In [6]:
x_train[train['부실판단'] == 0].describe()

,CASH FLOW 대 부채비율,당좌비율,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,이윤분배율,유형자산회전율,상장년수,PCR,쭈피처,도입기,성장기,성숙기,쇠퇴기
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02,128.000000,128.000000,128.000000,128.000000,128.000000
mean,20.767891,294.839453,27.524922,65.027187,2.417759,3.343864,3.646223,0.479306,-20.403672,4.717109,11.913281,-8.068720e+05,0.267252,0.257812,0.257812,0.281250,0.203125
std,40.494099,378.047818,23.097328,18.814115,13.239966,8.545632,15.170222,2.529254,175.186156,8.733986,7.235220,3.819671e+06,0.232296,0.439149,0.439149,0.451376,0.403906
min,-30.840000,19.750000,-42.750000,19.790000,-58.955600,-21.744800,-56.410800,-11.230100,-1507.750000,0.000000,4.000000,-2.892257e+07,0.020767,0.000000,0.000000,0.000000,0.000000
25%,-2.250000,98.500000,11.750000,52.000000,-0.937500,-0.527500,-1.302500,-0.557500,0.000000,1.000000,6.000000,-2.290970e+04,0.121640,0.000000,0.000000,0.000000,0.000000
50%,9.500000,164.000000,29.000000,67.000000,4.470000,3.300000,5.365000,0.350000,19.000000,2.000000,9.000000,5.403612e+03,0.193921,0.000000,0.000000,0.000000,0.000000
75%,24.250000,288.500000,43.000000,80.250000,8.807500,7.455000,11.210000,1.390000,36.000000,4.000000,16.250000,1.456317e+04,0.317266,1.000000,1.000000,1.000000,0.000000
max,204.000000,2092.450000,75.440000,95.810000,48.247500,48.710600,44.680300,10.308400,130.250000,80.160000,33.400000,2.166382e+05,1.584983,1.000000,1.000000,1.000000,1.000000


In [7]:
x_train[train['부실판단'] == 1].describe()

,CASH FLOW 대 부채비율,당좌비율,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,이윤분배율,유형자산회전율,상장년수,PCR,쭈피처,도입기,성장기,성숙기,쇠퇴기
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02,128.000000,128.000000,128.000000,128.000000,128.000000
mean,-5.220625,176.922891,11.638828,50.845078,-12.981158,-4.839901,-11.454908,2.537267,-74.228672,11.062578,17.278359,1.791169e+05,0.339613,0.257812,0.257812,0.281250,0.203125
std,33.812139,294.560373,28.141801,19.540335,21.013553,9.307686,19.691276,3.601243,176.214527,26.872569,8.201728,2.107737e+06,0.278308,0.439149,0.439149,0.451376,0.403906
min,-204.800000,11.800000,-63.650000,2.400000,-96.959500,-52.630000,-80.950000,-7.806000,-1217.390000,0.000000,4.000000,-1.166050e+07,0.060921,0.000000,0.000000,0.000000,0.000000
25%,-10.150000,58.750000,-9.000000,37.750000,-18.790000,-7.942500,-18.850000,0.517500,-81.750000,1.000000,10.000000,-1.880280e+04,0.135289,0.000000,0.000000,0.000000,0.000000
50%,-2.000000,94.000000,11.500000,50.000000,-8.355000,-3.125000,-5.325000,2.010000,-2.000000,3.000000,17.000000,4.773085e+03,0.257211,0.000000,0.000000,0.000000,0.000000
75%,6.000000,155.750000,27.250000,63.250000,0.055000,0.937500,1.250000,3.525000,0.250000,6.000000,22.000000,2.386216e+04,0.442381,1.000000,1.000000,1.000000,0.000000
max,109.000000,2258.940000,77.000000,95.000000,16.690000,13.207500,26.026400,17.295900,59.930000,160.000000,46.160000,1.307304e+07,1.430953,1.000000,1.000000,1.000000,1.000000


# standard scaling

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

labels = x_train.columns

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(data=x_train, columns = labels)
x_test = pd.DataFrame(data=x_test, columns = labels)

# modeling

In [9]:
default = pd.DataFrame()
best = pd.DataFrame()
validation = pd.DataFrame()

## LogisticRegression

### 기본 모델

In [26]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)


print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 스코어: {f1:.2f}')
print(f'ROC AUC 스코어: {roc_auc:.2f}')

# 결과 저장
result_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3)
}
default['LogisticRegression'] = result_logit
default

Accuracy: 0.766
Precision: 0.772
Recall: 0.772
F1 스코어: 0.77
ROC AUC 스코어: 0.77


,LogisticRegression
Accuracy,0.766
Precision,0.772
Recall,0.772
F1,0.772
ROC AUC,0.766


### 하이퍼 파라미터 조정

In [27]:
# 하이퍼파라미터 범위 지정
param_grid = {
    'C': np.arange(0, 10.0, 0.1),  # 규제 강도
    'penalty': ['l1', 'l2', 'elasticnet'],  # 규제 유형
    'solver': ['liblinear', 'saga']  # 최적화 알고리즘
}

model = LogisticRegression()
random_logit = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_logit.fit(x_train, y_train)

best_model = random_logit.best_estimator_

y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Logistic 하이퍼파라미터 조정 평가 지표")
print("Best Parameters:", random_logit.best_params_)
print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'ROC AUC Score: {roc_auc:.3f}')

# 파라미터 조정 결과 저장
result0_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3),
    'Best Parameters' : random_logit.best_params_
}
best['LogisticRegression'] = result0_logit
best

Fitting 7 folds for each of 99 candidates, totalling 693 fits


Logistic 하이퍼파라미터 조정 평가 지표
Best Parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.4}
Accuracy: 0.772
Precision: 0.786
Recall: 0.762
F1 Score: 0.774
ROC AUC Score: 0.772


,LogisticRegression
Accuracy,0.772
Precision,0.786
Recall,0.762
F1,0.774
ROC AUC,0.772
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ..."


### 최적 파라미터 검증

In [14]:
# 모델 생성
cv_logit_model = LogisticRegression(solver='liblinear', penalty='l1', C=0.4)

# Cross Validation
cv_accuracy = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_logit = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LogisticRegression'] = cv_result_logit
validation

CV_Accuracy_Scores: [0.78378378 0.59459459 0.72972973 0.86486486 0.80555556 0.91666667
 0.86111111]
CV_Precision_Scores: [0.77777778 0.57894737 0.73684211 0.9375     0.86666667 0.89473684
 0.80952381]
CV_Recall_Scores: [0.77777778 0.61111111 0.73684211 0.78947368 0.72222222 0.94444444
 0.94444444]
CV_F1_Scores: [0.77777778 0.59459459 0.73684211 0.85714286 0.78787879 0.91891892
 0.87179487]
CV_ROC/AUC: [0.80701754 0.64912281 0.84502924 0.9502924  0.89197531 0.9537037
 0.93518519]

=======교차검증 결과=======
CV_Accuracy_mean: 0.794
CV_Precision_mean: 0.800
CV_Recall_mean: 0.789
CV_F1_스코어_mean: 0.792
CV_ROC_AUC+스코어_mean: 0.862


## RandomForestClassifier

### 기본 모델

In [16]:
# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(x_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print("Random Forest 모델 평가 지표")
print(f'Accuracy: {accuracy_rf:.3f}')
print(f'Precision: {precision_rf:.3f}')
print(f'Recall: {recall_rf:.3f}')
print(f'F1 스코어: {f1_rf:.2f}')
print(f'ROC AUC 스코어: {roc_auc_rf:.2f}')

# 결과 저장
result_rf = {
    'Accuracy' : round(accuracy_rf, 3),
    'Precision' : round(precision_rf, 3),
    'Recall' : round(recall_rf, 3),
    'F1' : round(f1_rf, 3),
    'ROC AUC' : round(roc_auc_rf, 3)
}
default['RandomForestClassifier'] = result_rf
default

Random Forest 모델 평가 지표
Accuracy: 0.827
Precision: 0.825
Recall: 0.842
F1 스코어: 0.83
ROC AUC 스코어: 0.83


,LogisticRegression,RandomForestClassifier
Accuracy,0.766,0.827
Precision,0.772,0.825
Recall,0.772,0.842
F1,0.772,0.833
ROC AUC,0.766,0.827


### 하이퍼 파라미터 조정

In [19]:
random_search = {'n_estimators': np.arange(0, 200, 10),
                 'max_depth': np.arange(0, 9, 1),
                 'min_samples_split': np.arange(1, 9, 1),
                 'min_samples_leaf': np.arange(1, 9, 1)}

clf = RandomForestClassifier()
random_rf = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 99, 
                               cv = 7, verbose= 1, random_state= 101, n_jobs = -1)
random_rf.fit(x_train,y_train)
random_pf = random_rf.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("Random Forest 모델 하이퍼파라미터 조정 평가 지표")
print(random_rf.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_rf = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_rf.best_params_
}
best['RandomForestClassifier'] = result0_rf
best

Fitting 7 folds for each of 99 candidates, totalling 693 fits


Random Forest 모델 하이퍼파라미터 조정 평가 지표
{'n_estimators': 110, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': 8}
Accuracy: 0.772
Precision: 0.764
Recall: 0.802
F1 스코어: 0.783
ROC AUC 스코어: 0.771


,LogisticRegression,RandomForestClassifier
Accuracy,0.772,0.772
Precision,0.786,0.764
Recall,0.762,0.802
ROC AUC,0.772,0.771
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 110, 'min_samples_split': 4, ..."


### 최적 파라미터 검증

In [33]:
# 모델 생성
cv_model_rf = RandomForestClassifier(n_estimators=110, min_samples_split=4, min_samples_leaf=1, max_depth=8)
    # n_estimators=170, min_samples_split=8, min_samples_leaf=1, max_depth=27

# Cross Validation
cv_accuracy = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['RandomForestClassifier'] = cv_result_rf
validation

CV_Accuracy_Scores: [0.72972973 0.62162162 0.64864865 0.86486486 0.80555556 0.80555556
 0.86111111]
CV_Precision_Scores: [0.66666667 0.65217391 0.73333333 0.94117647 0.86666667 0.85
 0.89473684]
CV_Recall_Scores: [0.88888889 0.66666667 0.57894737 0.68421053 0.72222222 0.94444444
 0.94444444]
CV_F1_Scores: [0.74418605 0.71428571 0.64705882 0.8        0.78787879 0.91891892
 0.89473684]
CV_ROC/AUC: [0.8128655  0.70760234 0.78362573 0.96491228 0.87654321 0.96296296
 0.92283951]

=======교차검증 결과=======
CV_Accuracy_mean: 0.762
CV_Precision_mean: 0.801
CV_Recall_mean: 0.776
CV_F1_스코어_mean: 0.787
CV_ROC_AUC+스코어_mean: 0.862


,RandomForestClassifier
Accuracy,0.762
Precision,0.801
Recall,0.776
F1,0.787
ROC AUC,0.862


## AdaBoostClassifier

### 기본 모델

In [34]:
# AdaBoost 모델 생성 및 학습
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_adaboost = adaboost_model.predict(x_test)

accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("AdaBoost 모델 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 스코어: {f1_adaboost:.3f}')
print(f'ROC AUC 스코어: {roc_auc_adaboost:.3f}')

# 결과 저장
result_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3)
}
default['AdaBoostClassifier'] = result_adaboost
default

AdaBoost 모델 평가 지표:
Accuracy: 0.777
Precision: 0.813
Recall: 0.733
F1 스코어: 0.771
ROC AUC 스코어: 0.778


,LogisticRegression,AdaBoostClassifier
Accuracy,0.766,0.777
Precision,0.772,0.813
Recall,0.772,0.733
F1,0.772,0.771
ROC AUC,0.766,0.778


### 하이퍼 파라미터 조정

In [10]:
adaboost_model = AdaBoostClassifier()

param_grid = {
    'n_estimators': np.arange(1, 100, 10),  # 트리 개수
    'learning_rate': np.arange(0.001, 1, 0.001)  # 학습률
}

random_adaboost = RandomizedSearchCV(estimator=adaboost_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_adaboost.fit(x_train, y_train)

best_model = random_adaboost.best_estimator_

y_pred_adaboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("Best Parameters:", random_adaboost.best_params_)
print("AdaBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 Score: {f1_adaboost:.3f}')
print(f'ROC AUC Score: {roc_auc_adaboost:.3f}')

# 파라미터 조정 결과 저장
result0_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3),
    'Best Parameters' : random_adaboost.best_params_
}
best['AdaBoostClassifier'] = result0_adaboost
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits
Best Parameters: {'n_estimators': 21, 'learning_rate': 0.97}
AdaBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.817
Precision: 0.828
Recall: 0.812
F1 Score: 0.820
ROC AUC Score: 0.817


,AdaBoostClassifier
Accuracy,0.817
Precision,0.828
Recall,0.812
F1,0.82
ROC AUC,0.817
Best Parameters,"{'n_estimators': 21, 'learning_rate': 0.97}"


### 최적 파라미터 검증

In [37]:
# 모델 생성 
# AdaBoostClassifier 최적파라미터 적용 Crossvalidation
cv_model_adaboost = AdaBoostClassifier(n_estimators=30, learning_rate=0.501)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_adaboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['AdaBoostClassifier'] = cv_result_adaboost

CV_Accuracy_Scores: [0.72972973 0.59459459 0.7027027  0.86486486 0.83333333 0.86111111
 0.88888889]
CV_Precision_Scores: [0.69565217 0.57894737 0.75       0.88888889 0.875      0.84210526
 0.88888889]
CV_Recall_Scores: [0.88888889 0.61111111 0.63157895 0.84210526 0.77777778 0.88888889
 0.94444444]
CV_F1_Scores: [0.76190476 0.59459459 0.68571429 0.86486486 0.82352941 0.86486486
 0.91891892]
CV_ROC/AUC: [0.78362573 0.63450292 0.8245614  0.92982456 0.90432099 0.9367284
 0.93518519]

=======교차검증 결과=======
CV_Accuracy_mean: 0.782
CV_Precision_mean: 0.788
CV_Recall_mean: 0.798
CV_F1_스코어_mean: 0.788
CV_ROC_AUC+스코어_mean: 0.850


## BaggingClassifier

### 기본 모델

In [38]:
# Bagging 모델 생성 및 학습
bagging_model = BaggingClassifier()
bagging_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_bagging = bagging_model.predict(x_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Bagging 모델 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 스코어: {f1_bagging:.3f}')
print(f'ROC AUC 스코어: {roc_auc_bagging:.3f}')

# 결과 저장
result_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3)
}
default['BaggingClassifier'] = result_bagging
default

Bagging 모델 평가 지표:
Accuracy: 0.807
Precision: 0.862
Recall: 0.743
F1 스코어: 0.798
ROC AUC 스코어: 0.809


,LogisticRegression,AdaBoostClassifier,BaggingClassifier
Accuracy,0.766,0.777,0.807
Precision,0.772,0.813,0.862
Recall,0.772,0.733,0.743
F1,0.772,0.771,0.798
ROC AUC,0.766,0.778,0.809


### 하이퍼 파라미터 조정

In [40]:
bagging_model = BaggingClassifier()

param_grid = {
    'n_estimators': np.arange(10, 200, 10),
    'max_samples': np.arange(0.1, 1.0, 0.1),
    'max_features': np.arange(0.1, 1, 0.1)
}

random_bagging = RandomizedSearchCV(estimator=bagging_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_bagging.fit(x_train, y_train)

# 최적의 모델 저장
best_model = random_bagging.best_estimator_

# 최적의 모델로 예측 수행
y_pred_bagging = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Best Parameters:", random_bagging.best_params_)
print("Bagging 모델 하이퍼파라미터 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 Score: {f1_bagging:.3f}')
print(f'ROC AUC Score: {roc_auc_bagging:.3f}')

# 파라미터 조정 결과 저장
result0_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3),
    'Best Parameters' : random_bagging.best_params_
}
best['BaggingClassifier'] = result0_bagging
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits


Best Parameters: {'n_estimators': 180, 'max_samples': 0.7000000000000001, 'max_features': 0.6}
Bagging 모델 하이퍼파라미터 평가 지표:
Accuracy: 0.812
Precision: 0.802
Recall: 0.842
F1 Score: 0.821
ROC AUC Score: 0.811


,LogisticRegression,AdaBoostClassifier,BaggingClassifier
Accuracy,0.772,0.782,0.812
Precision,0.786,0.79,0.802
Recall,0.762,0.782,0.842
F1,0.774,0.786,0.821
ROC AUC,0.772,0.782,0.811
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000..."


### 최적 파라미터 검증

In [41]:
# 모델 생성 
# BaggingClassifier 최적파라미터 적용 Crossvalidation
cv_model_bagging = BaggingClassifier(n_estimators=180, max_samples=0.7, max_features=0.6)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_bagging = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['BaggingClassifier'] = cv_result_bagging

CV_Accuracy_Scores: [0.67307692 0.64705882 0.8627451  0.84313725 0.84313725]
CV_Precision_Scores: [0.61538462 0.66666667 0.875      0.90909091 0.85714286]
CV_Recall_Scores: [0.92307692 0.6        0.84       0.76923077 0.92307692]
CV_F1_Scores: [0.72727273 0.60869565 0.80851064 0.83333333 0.87272727]
CV_ROC/AUC: [0.76923077 0.76692308 0.93384615 0.90461538 0.93461538]

=======교차검증 결과=======
CV_Accuracy_mean: 0.774
CV_Precision_mean: 0.785
CV_Recall_mean: 0.811
CV_F1_스코어_mean: 0.770
CV_ROC_AUC+스코어_mean: 0.862


## SVC_linear

### 기본 모델

In [42]:
from sklearn.svm import SVC

# SVM 모델 생성 및 학습
svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_linear'] = result_svc_linear
default

SVM 모델 평가 지표:
Accuracy: 0.782
Precision: 0.796
Recall: 0.772
F1 스코어: 0.784
ROC AUC 스코어: 0.782


,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.766,0.777,0.807,0.782
Precision,0.772,0.813,0.862,0.796
Recall,0.772,0.733,0.743,0.772
F1,0.772,0.771,0.798,0.784
ROC AUC,0.766,0.778,0.809,0.782


### 하이퍼 파라미터 조정

In [63]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.1, 100, 0.1),  # 규제 매개변수
    'gamma': np.arange(0.0001, 1, 0.0001),  # 커널 계수
    'kernel': ['linear']  # 커널 타입
}

random_svc_linear = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_svc_linear.fit(x_train, y_train)

print("Best Parameters:", random_svc_linear.best_params_)
best_model = random_svc_linear.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_svc_linear.best_params_
}
best['SVC_linear'] = result0_svc_linear
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits
Best Parameters: {'kernel': 'linear', 'gamma': 0.4663, 'C': 0.9}
SVM 모델 평가 지표:
Accuracy: 0.782
Precision: 0.796
Recall: 0.772
F1 Score: 0.784
ROC AUC Score: 0.782


,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.782,0.812,0.782,0.772,0.792
Precision,0.786,0.79,0.802,0.796,0.811,0.841
Recall,0.762,0.782,0.842,0.772,0.723,0.733
F1,0.774,0.786,0.821,0.784,0.764,0.783
ROC AUC,0.772,0.782,0.811,0.782,0.773,0.793
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'linear', 'gamma': 0.4663, 'C': 0.9}","{'num_leaves': 174, 'n_estimators': 140, 'min_...","{'n_estimators': 450, 'learning_rate': 0.06440..."


,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.772,0.782,0.812,0.782
Precision,0.786,0.79,0.802,0.796
Recall,0.762,0.782,0.842,0.772
F1,0.774,0.786,0.821,0.784
ROC AUC,0.772,0.782,0.811,0.782
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'linear', 'gamma': 0.4663, 'C': 0.9}"


### 최적 파라미터 검증

In [46]:
# 모델 생성 
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_linear = SVC(kernel='linear', gamma=0.4663, C=0.9)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_linear = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_linear'] = cv_result_linear

CV_Accuracy_Scores: [0.67307692 0.66666667 0.8627451  0.82352941 0.78431373]
CV_Precision_Scores: [0.62857143 0.68181818 0.84615385 0.86956522 1.        ]
CV_Recall_Scores: [0.84615385 0.6        0.88       0.76923077 0.57692308]
CV_F1_Scores: [0.72131148 0.63829787 0.8627451  0.81632653 0.73170732]
CV_ROC/AUC: [0.77218935 0.74923077 0.92923077 0.87230769 0.97230769]

=======교차검증 결과=======
CV_Accuracy_mean: 0.762
CV_Precision_mean: 0.805
CV_Recall_mean: 0.734
CV_F1_스코어_mean: 0.754
CV_ROC_AUC+스코어_mean: 0.859


## SVC_rbf

### 기본모델

In [137]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_rbf'] = result_svc_rbf
default

SVM 모델 평가 지표:
Accuracy: 0.792
Precision: 0.812
Recall: 0.772
F1 스코어: 0.792
ROC AUC 스코어: 0.792


### 하이퍼 파라미터 조정

In [64]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.0001, 1, 0.0001),  # Reduce the maximum value
    'gamma': np.arange(0.0001, 0.1, 0.0001),  # Reduce the maximum value
    'kernel': ['rbf']
}

random_svc_rbf = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_svc_rbf.fit(x_train, y_train)

print("Best Parameters:", random_svc_rbf.best_params_)
best_model = random_svc_rbf.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_svc_rbf.best_params_
}
best['SVC_rbf'] = result0_svc_rbf
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits
Best Parameters: {'kernel': 'rbf', 'gamma': 0.0129, 'C': 0.7816000000000001}
SVM 모델 평가 지표:
Accuracy: 0.787
Precision: 0.781
Recall: 0.812
F1 Score: 0.796
ROC AUC Score: 0.786


,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier,XGBClassifier,SVC_rbf
Accuracy,0.772,0.782,0.812,0.782,0.772,0.792,0.787
Precision,0.786,0.79,0.802,0.796,0.811,0.841,0.781
Recall,0.762,0.782,0.842,0.772,0.723,0.733,0.812
F1,0.774,0.786,0.821,0.784,0.764,0.783,0.796
ROC AUC,0.772,0.782,0.811,0.782,0.773,0.793,0.786
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'linear', 'gamma': 0.4663, 'C': 0.9}","{'num_leaves': 174, 'n_estimators': 140, 'min_...","{'n_estimators': 450, 'learning_rate': 0.06440...","{'kernel': 'rbf', 'gamma': 0.0129, 'C': 0.7816..."


### 최적 파라미터 검증

In [50]:
# 모델 생성 
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_rbf = SVC(kernel='rbf', gamma=0.0129, C=0.7816)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rbf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_rbf'] = cv_result_rbf

CV_Accuracy_Scores: [0.63461538 0.74509804 0.8627451  0.84313725 0.90196078]
CV_Precision_Scores: [0.59459459 0.77272727 0.875      0.90909091 0.88888889]
CV_Recall_Scores: [0.84615385 0.68       0.84       0.76923077 0.92307692]
CV_F1_Scores: [0.6984127  0.72340426 0.85714286 0.83333333 0.90566038]
CV_ROC/AUC: [0.77514793 0.79538462 0.94153846 0.92307692 0.95230769]

=======교차검증 결과=======
CV_Accuracy_mean: 0.798
CV_Precision_mean: 0.808
CV_Recall_mean: 0.812
CV_F1_스코어_mean: 0.804
CV_ROC_AUC+스코어_mean: 0.877


## LGBMClassifier

In [54]:
# LGBM 모델 생성 및 학습
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_lgbm = lgbm_model.predict(x_test)

accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
precision_lgbm = precision_score(y_test, y_pred_lgbm)
recall_lgbm = recall_score(y_test, y_pred_lgbm)
f1_lgbm = f1_score(y_test, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)

print("LightGBM 모델 평가 지표:")
print(f'Accuracy: {accuracy_lgbm:.3f}')
print(f'Precision: {precision_lgbm:.3f}')
print(f'Recall: {recall_lgbm:.3f}')
print(f'F1 스코어: {f1_lgbm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_lgbm:.3f}')

# 결과 저장
result_lgbm = {
    'Accuracy' : round(accuracy_lgbm, 3),
    'Precision' : round(precision_lgbm, 3),
    'Recall' : round(recall_lgbm, 3),
    'F1' : round(f1_lgbm, 3),
    'ROC AUC' : round(roc_auc_lgbm, 3)
}
default['LGBMClassifier'] = result_lgbm
default

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 128, number of negative: 128
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001325 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 885
[LightGBM] [Info] Number of data points in the train set: 256, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier
Accuracy,0.766,0.777,0.807,0.782,0.802
Precision,0.772,0.813,0.862,0.796,0.844
Recall,0.772,0.733,0.743,0.772,0.752
F1,0.772,0.771,0.798,0.784,0.796
ROC AUC,0.766,0.778,0.809,0.782,0.803


### 파라미터 조정

In [55]:
param_grid = {
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1],
        'n_estimators': np.arange(10, 300, 10),
        'max_depth': np.arange(1, 10, 1),
        'num_leaves': np.arange(1, 200, 1),
        'min_child_samples': np.arange(1, 10, 1)
    }
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
# }

lgbm = LGBMClassifier()
random_lgbm = RandomizedSearchCV(lgbm, param_grid, cv = 7, n_jobs = -1, n_iter=99)
random_lgbm.fit(x_train,y_train)
random_pf = random_lgbm.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("LGBM 모델 하이퍼파라미터 조정 평가 지표")
print(random_lgbm.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_lgbm = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_lgbm.best_params_
}
best['LGBMClassifier'] = result0_lgbm
best

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 128, number of negative: 128
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 885
[LightGBM] [Info] Number of data points in the train set: 256, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier
Accuracy,0.772,0.782,0.812,0.787,0.772
Precision,0.786,0.79,0.802,0.781,0.811
Recall,0.762,0.782,0.842,0.812,0.723
F1,0.774,0.786,0.821,0.796,0.764
ROC AUC,0.772,0.782,0.811,0.786,0.773
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'rbf', 'gamma': 0.0129, 'C': 0.7816...","{'num_leaves': 174, 'n_estimators': 140, 'min_..."


### 최적 파라미터 검증

In [56]:
# 모델 생성 
# lgbm 최적파라미터 적용 Crossvalidation
cv_model_lgbm = LGBMClassifier(num_leaves=174, n_estimators=140, min_child_samples=7, max_depth=3, learning_rate=0.1)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_lgbm = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LGBMClassifier'] = cv_result_lgbm

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 102, number of negative: 102
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 742
[LightGBM] [Info] Number of data points in the train set: 204, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

#### 딥러닝

In [145]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# import tensorflow as tf

In [146]:
# model = Sequential()
# model.add(Dense(256, input_dim = 12, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation = 'sigmoid'))

In [147]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(x_train, y_train, epochs=200, batch_size=5)

# print("\n Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [148]:
# # 모델을 사용하여 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(x_test)
# y_pred = binarize(y_pred, threshold=0.5)  # 예측값을 0.5 임계값을 기준으로 이진 분류로 변환

# # 정확도(accuracy) 계산
# accuracy = accuracy_score(y_test, y_pred)
# print(f"accuracy: {accuracy:.4f}")

# # F1 점수(f1 score) 계산
# f1 = f1_score(y_test, y_pred)
# print(f"F1-Score: {f1:.4f}")

# # 재현율(recall) 계산
# recall = recall_score(y_test, y_pred)
# print(f"recall: {recall:.4f}")

# # 정밀도(precision) 계산
# precision = precision_score(y_test, y_pred)
# print(f"precision: {precision:.4f}")

## XGBoost

### 하이퍼 파라미터 조정

In [57]:
# 파라미터 조정
xgboost_model = XGBClassifier()

param_grid = {
    'n_estimators': np.arange(10, 500, 10),  # 트리 개수
    'learning_rate': np.arange(0.0001, 1, 0.0001)  # 학습률
}

random_search = RandomizedSearchCV(estimator=xgboost_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_xgboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
roc_auc_xgboost = roc_auc_score(y_test, y_pred_xgboost)

print("Best Parameters:", random_search.best_params_)
print("XGBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_xgboost:.3f}')
print(f'Precision: {precision_xgboost:.3f}')
print(f'Recall: {recall_xgboost:.3f}')
print(f'F1 Score: {f1_xgboost:.3f}')
print(f'ROC AUC Score: {roc_auc_xgboost:.3f}')

result0_xgboost = {
    'Accuracy' : round(accuracy_xgboost, 3),
    'Precision' : round(precision_xgboost, 3),
    'Recall' : round(recall_xgboost, 3),
    'F1' : round(f1_xgboost, 3),
    'ROC AUC' : round(roc_auc_xgboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['XGBClassifier'] = result0_xgboost
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits
Best Parameters: {'n_estimators': 450, 'learning_rate': 0.06440000000000001}
XGBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.792
Precision: 0.841
Recall: 0.733
F1 Score: 0.783
ROC AUC Score: 0.793


,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.782,0.812,0.787,0.772,0.792
Precision,0.786,0.79,0.802,0.781,0.811,0.841
Recall,0.762,0.782,0.842,0.812,0.723,0.733
F1,0.774,0.786,0.821,0.796,0.764,0.783
ROC AUC,0.772,0.782,0.811,0.786,0.773,0.793
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'rbf', 'gamma': 0.0129, 'C': 0.7816...","{'num_leaves': 174, 'n_estimators': 140, 'min_...","{'n_estimators': 450, 'learning_rate': 0.06440..."


### 최적 파라미터 검증

In [59]:
# 최적 파라미터 검증
xgboost_model = XGBClassifier(n_estimators=450, learning_rate=0.0644)

# Cross Validation
cv_accuracy = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='roc_auc')

print('=======교차검증 결과=======')
print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 평균값=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_xgboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['XGBClassifier'] = cv_result_xgboost

=======교차검증 결과=======
CV_Accuracy_Scores: [0.75       0.70588235 0.80392157 0.84313725 0.8627451 ]
CV_Precision_Scores: [0.6969697  0.72727273 0.82608696 0.875      0.95238095]
CV_Recall_Scores: [0.88461538 0.64       0.76       0.80769231 0.76923077]
CV_F1_Scores: [0.77966102 0.68085106 0.79166667 0.84       0.85106383]
CV_ROC/AUC: [0.78994083 0.74615385 0.90923077 0.89538462 0.92923077]

=======교차검증 평균값=======
CV_Accuracy_mean: 0.793
CV_Precision_mean: 0.816
CV_Recall_mean: 0.772
CV_F1_스코어_mean: 0.789
CV_ROC_AUC_mean: 0.854


# 결과

In [60]:
default

,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier
Accuracy,0.766,0.777,0.807,0.782,0.802
Precision,0.772,0.813,0.862,0.796,0.844
Recall,0.772,0.733,0.743,0.772,0.752
F1,0.772,0.771,0.798,0.784,0.796
ROC AUC,0.766,0.778,0.809,0.782,0.803


In [61]:
best

,LogisticRegression,AdaBoostClassifier,BaggingClassifier,SVC_linear,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.782,0.812,0.787,0.772,0.792
Precision,0.786,0.79,0.802,0.781,0.811,0.841
Recall,0.762,0.782,0.842,0.812,0.723,0.733
F1,0.774,0.786,0.821,0.796,0.764,0.783
ROC AUC,0.772,0.782,0.811,0.786,0.773,0.793
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'n_estimators': 30, 'learning_rate': 0.501}","{'n_estimators': 180, 'max_samples': 0.7000000...","{'kernel': 'rbf', 'gamma': 0.0129, 'C': 0.7816...","{'num_leaves': 174, 'n_estimators': 140, 'min_...","{'n_estimators': 450, 'learning_rate': 0.06440..."


In [62]:
validation

,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.762,0.782,0.774,0.762,0.798,0.762,0.793
Precision,0.801,0.788,0.785,0.805,0.808,0.783,0.816
Recall,0.776,0.798,0.811,0.734,0.812,0.765,0.772
F1,0.787,0.788,0.770,0.754,0.804,0.764,0.789
ROC AUC,0.862,0.850,0.862,0.859,0.877,0.840,0.854
